<a href="https://colab.research.google.com/github/geocarvalho/uni-proj/blob/master/IN1164/project_1/PSO_and_ABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação: 
- PSO padrão e uma variação.
- ABC (ou Firefly) padrão e uma variação.


Testar em 6 funções de benchmark com variação de atributos de 10, 20 e 50.
Funções:
- [Ackley function](http://benchmarkfcns.xyz/benchmarkfcns/ackleyfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/ackley.html)
- [Alpine function](http://benchmarkfcns.xyz/benchmarkfcns/alpinen1fcn.html)
- [Schwefel Function](http://benchmarkfcns.xyz/benchmarkfcns/schwefelfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/schwefel.html)
- [Happy Cat Function](http://benchmarkfcns.xyz/benchmarkfcns/happycatfcn.html) 
- [Brown function](http://benchmarkfcns.xyz/benchmarkfcns/brownfcn.html)
- [Exponential function](http://benchmarkfcns.xyz/benchmarkfcns/exponentialfcn.html)

Para funções olhar o site: http://benchmarkfcns.xyz/

# Funções de fitness


In [2]:
import math
import random
import numpy as np

def simple_func(position):
  """ Simple function that models the problem """
  return position[0]**2 + position[1]**2 + 1

def sphere_func(positions):
  """ Sphere function 
  http://benchmarkfcns.xyz/benchmarkfcns/spherefcn.html"""
  c = np.array(positions)
  x = c**2
  return np.sum(x)

def ackley_func(positions):
  """ Ackley function"""
  c = np.array(positions)
  firstSum = np.sum(c**2.0)
  secondSum = np.sum(np.cos(2.0*math.pi*c))
  n = float(len(positions))
  return -20.0*math.exp(-0.2*math.sqrt(firstSum/n)) - math.exp(secondSum/n) + 20 + math.e

def alpine_func(positions):
  """ Alpine function """
  c = np.array(positions)
  scores = np.sum(abs(c * np.sin(c) + 0.1*c))
  return scores

def schwefel_func(positions):
  """F7 Schwefel's function
  multimodal, asymmetric, separable"""
  c = np.array(positions)
  n = len(c)
  alpha = 418.982887
  fitness = np.sum(c * np.sin(np.sqrt(abs(c))))
  return alpha * n - fitness

def happy_cat_func(positions):
  """ Happy Cat function """
  alpha = 0.5
  c = np.array(positions)
  n = len(c)
  x2 = np.sum(c*c)
  scores = (((x2 - n)**2)**alpha + (0.5*x2 + np.sum(c)))/ (n + 0.5)
  return scores

def brown_func(positions):
  """ Brown function """
  c = np.array(positions)
  n = len(c)
  x = c**2
  scores = 0
  for i in range(n-1):
    scores = scores + x[i]**(x[i+1] + 1) + x[i+1]**(x[i]+1)
  return scores

def exponential_func(positions):
  """ Exponential function """
  c = np.array(positions)
  x2 = c**2
  scores = -np.exp(-0.5 * np.sum(x2))
  return scores


# PSO padrão

* [1995 Particle swarm optimization](https://ieeexplore.ieee.org/abstract/document/488968)
* [1998 A Modified Particle Swarm Optimizer](https://ieeexplore.ieee.org/document/699146)

In [ ]:
def PSO(problem, dimension, var_min, var_max, n_iterations, n_particles,
        wdamp, w, c1, c2, show_iter):
  """ PSO algorithm """
  # Inicialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = particle_position_vector
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.array([float('inf'), float('inf')])
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0

  # Start iterations
  while iteration < n_iterations:
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = particle_position_vector[p]
      
      # Calculate gbest
      if gbest_fitness_value > fitness_candidate:
        gbest_fitness_value = fitness_candidate
        gbest_position = particle_position_vector[p]
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      particle_position_vector[p] = new_position

    iteration += 1
    w *= wdamp

  # print("The best position is: ", gbest_position, " with value of ", gbest_fitness_value,
  #       " in iteration number ", iteration)
  return gbest_position, gbest_fitness_value

In [ ]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": -10, 
              "var_max": 10, "n_iterations": 30, "n_particles": 50,
              "wdamp": 0.99, "w": 1, "c1": 2, "c2": 2, "show_iter": False}
    gbest_pos, gbest_value = PSO(**kwargs)
    print("For dimension %s using the %s, the gbest was: %s" % (dim, name, gbest_value))

For dimension 10 using the Ackley function, the gbest was: 3.915421669265641
For dimension 10 using the Alpine function, the gbest was: 3.7860278799787066
For dimension 10 using the Schwefel's function, the gbest was: 4149.781805058077
For dimension 10 using the Happy Cat function, the gbest was: 0.6541087149515272
For dimension 10 using the Brown function, the gbest was: 33.463181132317594
For dimension 10 using the Exponential function, the gbest was: -0.010342054296966791


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in double_scalars


For dimension 20 using the Ackley function, the gbest was: 6.436154939649434
For dimension 20 using the Alpine function, the gbest was: 13.763747796540372
For dimension 20 using the Schwefel's function, the gbest was: 8298.608960497542
For dimension 20 using the Happy Cat function, the gbest was: 1.9717850518919877
For dimension 20 using the Brown function, the gbest was: 14602.903672688088
For dimension 20 using the Exponential function, the gbest was: -5.715374523596763e-12
For dimension 50 using the Ackley function, the gbest was: 8.521931481667345
For dimension 50 using the Alpine function, the gbest was: 53.798887881288856
For dimension 50 using the Schwefel's function, the gbest was: 20773.04886869769
For dimension 50 using the Happy Cat function, the gbest was: 2.9374231654057277
For dimension 50 using the Brown function, the gbest was: 5708779502.177752
For dimension 50 using the Exponential function, the gbest was: -6.745876194841163e-45


# Variação do PSO

* [2002 The particle swarm - explosion, stability, and convergence in a multidimensional complex space](https://ieeexplore.ieee.org/abstract/document/985692)

> Constriction coefficients

$\chi = \frac{2k}{|2 - \phi - \sqrt{\phi^2 - 4\phi}|}$

* Onde $0\leq k \leq 1$ e $\phi_1, \phi_2 = 2.05$;
* Valor de $k=1$;
* Substituições $w = \chi$, $c_1 = \chi.\phi_1$ e $c_2 = \chi.\phi_2$.
* Além disso, $\phi = \phi_1 + \phi_2 \geq 4$

In [ ]:
def PSO_cc(problem, dimension, var_min, var_max, n_iterations, n_particles,
        wdamp, w, c1, c2, show_iter):
  """ PSO algorithm """
  # Inicialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = particle_position_vector
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.array([float('inf'), float('inf')])
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0

  # Start iterations
  while iteration < n_iterations:
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = particle_position_vector[p]
      
      # Calculate gbest
      if gbest_fitness_value > fitness_candidate:
        gbest_fitness_value = fitness_candidate
        gbest_position = particle_position_vector[p]
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      particle_position_vector[p] = new_position

    iteration += 1
    w *= wdamp

  # print("The best position is: ", gbest_position, " with value of ", gbest_fitness_value,
  #       " in iteration number ", iteration)
  return gbest_position, gbest_fitness_value

In [ ]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

phi1 = 2.05
phi2 = 2.05
phi = phi1 + phi2
kappa = 1
chi = 2*kappa/(abs(2-phi-math.sqrt(phi**2 - 4*phi)))

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": -10, 
              "var_max": 10, "n_iterations": 1000, "n_particles": 50,
              "wdamp": 1, "w": chi, "c1": chi*phi1, "c2": chi*phi2, "show_iter": False}
    gbest_pos, gbest_value = PSO_cc(**kwargs)
    print("For dimension %s using the %s, the gbest was: %s" % (dim, name, gbest_value))

For dimension 10 using the Ackley function, the gbest was: 5.482614393629319
For dimension 10 using the Alpine function, the gbest was: 5.284884444645204
For dimension 10 using the Schwefel's function, the gbest was: 4162.266288272899
For dimension 10 using the Happy Cat function, the gbest was: 1.2984748809579425
For dimension 10 using the Brown function, the gbest was: 21.2898701014414
For dimension 10 using the Exponential function, the gbest was: -8.492091376944449e-06
For dimension 20 using the Ackley function, the gbest was: 7.438340349306808
For dimension 20 using the Alpine function, the gbest was: 19.522356352826066
For dimension 20 using the Schwefel's function, the gbest was: 8318.120414521023
For dimension 20 using the Happy Cat function, the gbest was: 1.7546537364132178
For dimension 20 using the Brown function, the gbest was: 945.6573796677054
For dimension 20 using the Exponential function, the gbest was: -1.1184914514302178e-09
For dimension 50 using the Ackley functio

# Outras referências

* [A novel particle swarm optimization algorithm with adaptive inertia weight](https://www.sciencedirect.com/science/article/abs/pii/S156849461100055X)
* [The particle swarm optimization algorithm: convergence analysis and parameter selection](https://www.sciencedirect.com/science/article/abs/pii/S0020019002004477)
* [An Overview of Particle Swarm Optimization Variants](https://www.sciencedirect.com/science/article/pii/S1877705813001823)
* [2020 Population size in Particle Swarm Optimization](https://www.sciencedirect.com/science/article/pii/S2210650220303710)

In [38]:
def fitness_calc(value):
  """ Calculate the fitness based on the object function result """
  if value >= 0:
    return 1/(1+value)
  else:
    return 1 + abs(value)

# Problem
dimension = 4
fitness_func = sphere_func
var_min = 0.0
var_max = 10.0

# Fix parameters
s = 10
t = 10
limit = 1
phi = 0.81 # how does this work?

employed_bees = int(s/2)
onlooker_bees = int(s/2)
food_source = int(s/2)

# Initialization
## The food source population with 4 variables, create random value 0<x<10
population = np.random.uniform(var_min,var_max,(food_source, dimension)) # not randint
print(population)
## Calculate the object function for each food source
obj = np.apply_along_axis(fitness_func, 1, population)
## Calculate randintfitness of the population
print(obj)
fit = np.array([fitness_calc(value) for value in obj])
print(fit)
## Generate initial trial vector
trial = np.zeros(food_source)
print(trial)

# Employed bee phase 4:15
print("\nEmployed bee phase")
for num, food in enumerate(population, start=0):
  ## Select a random variable to change
  var_index_lst = list(range(dimension))
  var_index = random.choice(var_index_lst)
  food_var = food[var_index]
  ## Select a random partner
  partner_index_lst = list(range(food_source))
  partner_index_lst.remove(num)
  partner_index = random.choice(partner_index_lst)
  partner = population[partner_index]
  partner_var = partner[var_index]
  print(food)
  print(food_var, partner, partner_var)
  ## Create a new food location
  x_new = food_var + (phi*(food_var - partner_var))
  ## And if the value is ok? how we put the value in the matrix?
  ## Evaluate the fitness
  new_obj = fitness_func(food[var_index])
  new_fit = fitness_calc(new_obj)
  print("New fit: ", new_fit)
  ## Perform greedy selection
  if new_fit > fit[num]:
    ## Check if the new location violates the lower bound (0<=x<=10)
    if not var_min <= x_new <= var_max:
      if x_new < var_min:
        food[var_index] = var_min
      else:
        food[var_index] = var_max
    else:
      food[var_index] = x_new
    fit[num] = new_fit
    obj[num] = new_obj
    trial[num] = 0.0
  else:
    trial[num] += 1.0
# Calculate the probability values 15:46

  print(x_new)
  print(food)
  print('end----\n')
print(population)
print(obj)
print(fit)
print(trial)

[[1.56162365e+00 7.43606292e+00 7.70253067e+00 6.40824764e+00]
 [8.94983197e-03 8.45099921e+00 5.49580246e-01 7.14241455e+00]
 [8.33870434e+00 3.69651150e+00 3.89217632e+00 9.03063103e+00]
 [8.59798328e+00 2.32380329e+00 7.42094695e+00 7.40858013e+00]
 [8.79407272e+00 9.24601241e+00 8.73940682e+00 2.26430954e+00]]
[158.1283167  122.73559184 179.89952059 189.28289147 244.32878961]
[0.00628424 0.00808175 0.00552793 0.00525533 0.00407616]
[0. 0. 0. 0. 0.]

Employed bee phase
[1.56162365 7.43606292 7.70253067 6.40824764]
6.408247641623133 [8.79407272 9.24601241 8.73940682 2.26430954] 2.2643095409788048
New fit:  0.023772372212443573
9.764837503145039
[1.56162365 7.43606292 7.70253067 9.7648375 ]
end----

[0.00894983 8.45099921 0.54958025 7.14241455]
0.5495802464522948 [8.79407272 9.24601241 8.73940682 2.26430954] 8.739406819293347
New fit:  0.7680264757779754
-6.084179277548959
[0.00894983 8.45099921 0.         7.14241455]
end----

[8.33870434 3.6965115  3.89217632 9.03063103]
9.0306310310

In [ ]:
np.zeros(shape=(1,food_s))

array([[0., 0., 0., 0., 0.]])

In [20]:
lista = list(range(dimension))
print(lista)
num = random.choice(lista)
lista.remove(num)
print(lista)
print(num)

[0, 1, 2, 3]
[0, 1, 2]
3
